# Linear Optimization - Advanced Production Planning

Shipping example

In [1]:
# Reference:  https://coin-or.github.io/pulp/

# Dependencies 
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum

In [2]:
# Parameters
customer_list= ['Australia', 'Sweden', 'Brazil']
distribution_list = ['DC1', 'DC2']
factory_list= ['Factory1', 'Factory2']
product_list= ['Chair', 'Table', 'Bed']
# capacity = {'Factory1':500, 'Factory2': 500}


In [3]:

# create combinations to be varied
dist_paths_keys = [(f,p,d,c) for f in factory_list for p in product_list for d in distribution_list for c in customer_list] 
factory_active_keys = [(f,p) for f in factory_list for p in product_list]
inbound_keys = [(f,p,d) for f in factory_list for p in product_list for d in distribution_list]
outbound_keys = [(c,d,p) for c in customer_list for d in distribution_list for p in product_list]

In [4]:
# Dataframes - normally read the data in but here I will create manually

# Demand Table
country_dict = {'Country': customer_list}
demand_data = {'Chair':[50000,12000,30000], 'Table': [30000,80000,60000], 'Bed':[45000,40000,175000]}
data = {**country_dict, **demand_data}
demand = pd.DataFrame(data).set_index('Country')

# Production Capacity
capacity_data = {'Factory':factory_list,
                 'Chair':[100000,100000],
                 'Table':[100000,100000],
                 'Bed':[130000,130000]
                 }
capacity = pd.DataFrame(capacity_data).set_index('Factory')
# Cost table
production_cost_data = {'Factory':['Factory1']*2 + ['Factory2']*2, 
             'Type':['Variable','Fixed']*2,
             'Chair':[50,30000,80,25000], 
             'Table':[60,25000,90,40000], 
             'Bed':[70,50000,90,40000]}

production_cost = pd.DataFrame(production_cost_data).set_index(['Factory','Type'])

# DC inbound fees table
inbound_fees_data = { 'Factory':['Factory1', 'Factory1', 'Factory2', 'Factory2'],
                      'DC':distribution_list*2,
                      'Chair':[10,4,2,10],
                      'Table':[20,5,3,12],
                      'Bed':[6,5,4,15]
                     }
inbound_fees = pd.DataFrame(inbound_fees_data).set_index(['Factory','DC'])

# DC outbound fees table
outbound_fees_data = { 'Country':['Australia']*2 + ['Sweden']*2 + ['Brazil']*2,
                      'DC':distribution_list*3,
                      'Chair':[8,7,7,4,8,10],
                      'Table':[9,6,6,5,9,12],
                      'Bed':[10,12,12,6,10,15]
                     }
outbound_fees = pd.DataFrame(outbound_fees_data).set_index(['Country','DC'])

# Show tables  
print('Demand Table\n', demand)
print('\n', '--'*20, '\n')
print('Capacity Table\n', capacity)
print('\n', '--'*20, '\n')
print('Production Cost Table\n', production_cost)
print('\n', '--'*20, '\n')


Demand Table
            Chair  Table     Bed
Country                        
Australia  50000  30000   45000
Sweden     12000  80000   40000
Brazil     30000  60000  175000

 ---------------------------------------- 

Capacity Table
            Chair   Table     Bed
Factory                         
Factory1  100000  100000  130000
Factory2  100000  100000  130000

 ---------------------------------------- 

Production Cost Table
                    Chair  Table    Bed
Factory  Type                         
Factory1 Variable     50     60     70
         Fixed     30000  25000  50000
Factory2 Variable     80     90     90
         Fixed     25000  40000  40000

 ---------------------------------------- 



In [5]:

print('Inbound Cost Table\n', inbound_fees)
print('\n', '--'*20, '\n')
print('Outbound Cost Table\n', outbound_fees)
print('\n', '--'*20, '\n')

Inbound Cost Table
               Chair  Table  Bed
Factory  DC                    
Factory1 DC1     10     20    6
         DC2      4      5    5
Factory2 DC1      2      3    4
         DC2     10     12   15

 ---------------------------------------- 

Outbound Cost Table
                Chair  Table  Bed
Country   DC                    
Australia DC1      8      9   10
          DC2      7      6   12
Sweden    DC1      7      6   12
          DC2      4      5    6
Brazil    DC1      8      9   10
          DC2     10     12   15

 ---------------------------------------- 



In [6]:
# # replaces keys and cost variables from above
# cost_combos={}
# for product in cost.columns:
#     column_dictionary = dict(cost[product])
#     for dict_key, value in column_dictionary.items():
#         new_key = (dict_key[0], product, dict_key[1])
#         cost_combos[new_key] = value

# cost_keys_pd = list(cost_combos.keys())

In [6]:
# Create linear programming model
model= LpProblem('distribution',LpMinimize)

# create model variable (dictionary of unique keys (tuples) and corresponding values)
# cat acts as a constraint to keep values as integer values
prod = LpVariable.dicts(name='path', indices=dist_paths_keys, lowBound=0, upBound=None, cat='Integer')
factory_active = LpVariable.dicts(name='active', indices=factory_active_keys, lowBound=0, upBound=None, cat='Binary')
inbound = LpVariable.dicts(name='inbound', indices=inbound_keys, lowBound=0, upBound=None, cat='Integer')
outbound = LpVariable.dicts(name='outbound', indices=outbound_keys, lowBound=0, upBound=None, cat='Integer')

# name – The prefix to the name of each LP variable created
# indices – A list of strings of the keys to the dictionary of LP variables, and the main part of the variable name itself
# lowBound – The lower bound on these variables’ range. Default is negative infinity
# upBound – The upper bound on these variables’ range. Default is positive infinity
# cat – The category these variables are in, Integer or Continuous(default)
# indexs – (deprecated) Replaced with indices parameter

In [7]:
# # replaces 
# demand_combos={}
# for product in demand.columns:
#     column_dictionary = dict(demand[product])
#     for dict_key, value in column_dictionary.items():
#         new_key = (product, dict_key)
#         demand_combos[new_key] = value

# demand_keys_pd = list(demand_combos.keys())

In [8]:
# create model objective that sums the product of cost and product sent (aka the model variable :: var)
model+= lpSum(prod[(f,p,d,c)]*production_cost.loc[(f,'Variable'),p] + 
              production_cost.loc[(f,'Fixed'),p]*factory_active[(f,p)] +
              prod[(f,p,d,c)]*inbound_fees.loc[(f,d),p] +
              prod[(f,p,d,c)]*outbound_fees.loc[(c,d),p]
              for f in factory_list for p in product_list for d in distribution_list for c in customer_list )


In [9]:
# create model constraint where Production < Capacity
for f in factory_list:
   for p in product_list:
      model += lpSum(prod[(f,p,d,c)] for d in distribution_list for c in customer_list ) <= capacity.loc[f,p]
               
# for p in product_list:
#    for d in distribution_list:
#       for c in customer_list:
#          model += lpSum(prod[('Factory2',p,d,c)]) <= capacity.loc['Factory2',p]

In [10]:
# Create constraint for Production Available <= 0
for f in factory_list:
    for p in product_list:
        model += (lpSum(prod[(f,p,d,c)] for c in customer_list for d in distribution_list) - 
                    factory_active[(f,p)]*capacity.loc[f,p]) <= 0


In [11]:
# Create constraint for Demand forecast < Delivered units
for c in customer_list:
    for p in product_list:
        model += (demand.loc[c,p] - lpSum(outbound[(c,d,p)] for d in distribution_list)) <= 0

In [12]:
# Create supply constraints
model += (lpSum(prod[('Factory1',p,d,c)] for c in customer_list for d in distribution_list for p in product_list) - 
            lpSum(inbound[('Factory1',p,d)] for p in product_list for d in distribution_list)) == 0

model += (lpSum(prod[('Factory2',p,d,c)] for c in customer_list for d in distribution_list for p in product_list) - 
            lpSum(inbound[('Factory2',p,d)] for p in product_list for d in distribution_list)) == 0  

model += (lpSum(inbound[(f,p,'DC1')] for f in factory_list for p in product_list) - 
            lpSum(outbound[(c,'DC1',p)] for c in customer_list for p in product_list)) == 0  

model += (lpSum(inbound[(f,p,'DC2')] for f in factory_list for p in product_list) - 
            lpSum(outbound[(c,'DC2',p)] for c in customer_list for p in product_list)) == 0  

model += (lpSum(prod[(f,'Chair',d,c)] for f in factory_list for d in distribution_list for c in customer_list) - 
            lpSum(inbound[(f,'Chair',d)] for f in factory_list for d in distribution_list)) == 0

model += (lpSum(prod[(f,'Table',d,c)] for f in factory_list for d in distribution_list for c in customer_list) - 
            lpSum(inbound[(f,'Table',d)] for f in factory_list for d in distribution_list)) == 0

model += (lpSum(prod[(f,'Bed',d,c)] for f in factory_list for d in distribution_list for c in customer_list) - 
            lpSum(inbound[(f,'Bed',d)] for f in factory_list for d in distribution_list)) == 0

model += (lpSum(inbound[(f,'Chair',d)] for f in factory_list for d in distribution_list) - 
            lpSum(outbound[(c,d,'Chair')] for c in customer_list for d in distribution_list)) == 0  

model += (lpSum(inbound[(f,'Table',d)] for f in factory_list for d in distribution_list) - 
            lpSum(outbound[(c,d,'Table')] for c in customer_list for d in distribution_list)) == 0  

model += (lpSum(inbound[(f,'Bed',d)] for f in factory_list for d in distribution_list) - 
            lpSum(outbound[(c,d,'Bed')] for c in customer_list for d in distribution_list)) == 0  

model += (lpSum(inbound[(f,'Chair','DC1')] for f in factory_list)  - 
            lpSum(outbound[(c,'DC1','Chair')] for c in customer_list)) == 0  

model += (lpSum(inbound[(f,'Table','DC1')] for f in factory_list)  - 
            lpSum(outbound[(c,'DC1','Table')] for c in customer_list)) == 0  

model += (lpSum(inbound[(f,'Bed','DC1')] for f in factory_list)  - 
            lpSum(outbound[(c,'DC1','Bed')] for c in customer_list)) == 0  

model += (lpSum(inbound[(f,'Chair','DC2')] for f in factory_list)  - 
            lpSum(outbound[(c,'DC2','Chair')] for c in customer_list)) == 0  

model += (lpSum(inbound[(f,'Table','DC2')] for f in factory_list)  - 
            lpSum(outbound[(c,'DC2','Table')] for c in customer_list)) == 0  

model += (lpSum(inbound[(f,'Bed','DC2')] for f in factory_list)  - 
            lpSum(outbound[(c,'DC2','Bed')] for c in customer_list)) == 0  

In [13]:
# run the model
solution = model.solve()
solution


1

In [14]:
# Minimized Cost (objective function)
model.objective.value()

44426000.0

In [29]:
pd.DataFrame([list(i) + [int(prod[i].varValue)] for i in prod], columns=['Factory', 'Product', 'DC','Country', 'Forecast Quantity']).drop(['DC','Country'], axis=1)

,Factory,Product,Forecast Quantity
0,Factory1,Chair,0
1,Factory1,Chair,0
2,Factory1,Chair,0
3,Factory1,Chair,0
4,Factory1,Chair,92000
5,Factory1,Chair,0
6,Factory1,Table,0
7,Factory1,Table,0
8,Factory1,Table,0
9,Factory1,Table,0


In [30]:
# Display results
pd.DataFrame([list(i) + [int(factory_active[i].varValue)] for i in factory_active], columns=['Factory', 'Product', 'Active'])

,Factory,Product,Active
0,Factory1,Chair,1
1,Factory1,Table,1
2,Factory1,Bed,1
3,Factory2,Chair,0
4,Factory2,Table,1
5,Factory2,Bed,1


In [31]:
# Display results
pd.DataFrame([list(i) + [int(inbound[i].varValue)] for i in inbound], columns=['Factory', 'Product', 'DC', 'Quantity'])

,Factory,Product,DC,Quantity
0,Factory1,Chair,DC1,92000
1,Factory1,Chair,DC2,0
2,Factory1,Table,DC1,0
3,Factory1,Table,DC2,0
4,Factory1,Bed,DC1,230000
5,Factory1,Bed,DC2,0
6,Factory2,Chair,DC1,0
7,Factory2,Chair,DC2,0
8,Factory2,Table,DC1,170000
9,Factory2,Table,DC2,0


In [32]:
# Display results
pd.DataFrame([list(i) + [int(outbound[i].varValue)] for i in outbound], columns=['Customer', 'DC','Product', 'Quantity'])

,Customer,DC,Product,Quantity
0,Australia,DC1,Chair,50000
1,Australia,DC1,Table,30000
2,Australia,DC1,Bed,45000
3,Australia,DC2,Chair,0
4,Australia,DC2,Table,0
5,Australia,DC2,Bed,0
6,Sweden,DC1,Chair,12000
7,Sweden,DC1,Table,80000
8,Sweden,DC1,Bed,40000
9,Sweden,DC2,Chair,0


**Notes**
Product Table looks as expected
Inbound and Outbound numbers are a bit different than my Excel model but I am leaning towards my Excel modeling having an error
* The reason my excel is has a problem is that my constraint that Factory 2 does not provide Chairs is violated in the outputs.  This constrain is correctly implemented here.